# Kolla om CatBoost ranking kan ge något

In [ ]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils


In [ ]:
# !pip3 install sklearn_pandas==2.0.4
# !pip3 install catboost==0.24.4
# !pip uninstall scikit-learn -y
# !pip3 install scikit-learn==0.24.1 
# import sklearn
# sklearn.show_versions()

In [ ]:
def comp_new_features(df):
    ### delta ###
    def delta(dat1, dat2): # delta är dat1-dat2
        dat1 = pd.to_datetime(dat1)
        dat2 = pd.to_datetime(dat2)
        delta= dat1-dat2
        return delta.dt.days
    
    df['delta0']=delta(df.datum.copy(),df.h1_dat.copy())
    df['delta1']=delta(df.h1_dat, df.h2_dat)
    df['delta2']=delta(df.h2_dat, df.h3_dat)
    df['delta3']=delta(df.h3_dat, df.h4_dat)
    df['delta4']=delta(df.h4_dat, df.h5_dat)
    
    ### performance ###
    df['h1_perf'] = (30-df.h1_plac*2)*df.h1_pris
    df['h2_perf'] = (30-df.h2_plac*2)*df.h2_pris
    df['h3_perf'] = (30-df.h3_plac*2)*df.h3_pris
    df['h4_perf'] = (30-df.h4_plac*2)*df.h4_pris
    df['h5_perf'] = (30-df.h5_plac*2)*df.h5_pris

    return df

In [ ]:
def check_unique(df):
    for feat in df:
        if len(df[feat].unique())==1:
            df.drop(feat,axis=1,inplace=True)
    return df       
   

In [ ]:
def check_corr(df):
    df.corr()
    return df

In [ ]:
def remove_features(df,remove_mer=[]):
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df.drop(['avd','vodds','podds','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True)
    df.drop(remove_mer,axis=1,inplace=True)
    
    df=check_unique(df.copy())
    df=check_corr(df.copy())
    return df

In [ ]:
### Spara max av Recall, F1, AUC etc till efter learning
def spara(model,test_pool,results):
    dic=model.eval_metrics(test_pool,['Recall','F1','AUC','Accuracy','Precision'])
    dic['best']=model.get_feature_importance(prettified=True)['Feature Id'][0]
    dic['worst']=model.get_feature_importance(prettified=True)['Feature Id'].iloc[-1]
    results.loc[len(results),:]=[None,None,None,None,None,None,None]
    results.loc[len(results)-1,:] = [np.max(dic['Recall']),np.max(dic['F1']),np.max(dic['AUC']),np.max(dic['Accuracy']),np.max(dic['Precision']),dic['best'],dic['worst']]
    print('       Recall  F1   AUC   Precision Accuracy')
    print('mean:',round(np.mean(results['Recall']),4),round(np.mean(results['F1']),4),round(np.mean(results['AUC']),4),
          round(np.mean(results['Accuracy']),4),round(np.mean(results['Precision']),4))
    print('std: ',round(np.std(results['Recall']),4),round(np.std(results['F1']),4),round(np.std(results['AUC']),4),
          round(np.std(results['Accuracy']),4),round(np.std(results['Precision']),4))
    print(f"best={dic['best']} worst={dic['worst']}")
    return results


In [ ]:
def handle_cat_features(df,cat):
    # for c in cat:
    #     df.loc[df[c].isna()][c] = 'UNK'

    df['start']=(df.start=='AUTOSTART').astype(bool)
    return df


### split så att allt fram till split_dat blir train och endast en split_dat (en omgång) blir test ###
def one_split(df,split_dat):

    print(f'len(df)={len(df)}')
    
    X_train = df.loc[df.datum<fr_dat].copy()
    y_train = X_train.plac==1
    X_train.drop(['datum','plac'],axis=1,inplace=True)
    X_test = df.loc[df.datum==fr_dat].copy()
    y_test = X_test.plac==1
    X_test.drop(['datum','plac'],axis=1,inplace=True)
    return X_train,X_test,y_train,y_test
        

In [ ]:

def run_catB(model,train_pool,test_pool): 
    model.fit(train_pool,eval_set=test_pool, use_best_model=True, verbose=50) 
    model

def estimator_loop(df,cat_features): 
    results = pd.DataFrame(columns=['Recall','AUC','F1','Precision','Accuracy','best','worst'])
    
    model = CatBoostClassifier(iterations= 1000, auto_class_weights= 'Balanced', 
            eval_metric= 'Recall', loss_function= 'Logloss', early_stopping_rounds= 100,
            random_seed= 2021)

    alla_datum = df.datum.unique()
    alla_datum = np.sort(alla_datum)
    cut_ix = int(len(alla_datum)*0.75)
    print(len(alla_datum[cut_ix:]))
    for nr,datum in enumerate(alla_datum[cut_ix:]):
        print(nr,'av',len(alla_datum[cut_ix:]))
        X_train,X_test,y_train,y_test = one_split(df,datum)
    
        # print('X_train',X_train,'\ny_train',y_train)
        train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
        test_pool = Pool(data=X_test, label=y_test,cat_features=cat_features)
        run_catB(model,train_pool, test_pool)

        results=spara(model,test_pool,results)
        
    return results


In [ ]:
def feature_types(X):
    cat_features = X.select_dtypes('O').columns.to_list()
    int_features = X.select_dtypes('int64').columns.to_list()
    float_features = X.select_dtypes(float).columns.to_list()
    
    return cat_features,int_features,float_features

In [ ]:
df=pd.read_csv('all_data.csv')
results = pd.DataFrame(columns=['AUC','Accuracy','F1','best','worst'])

#### min egen cv loop ####
import time
start = time.time()

# sortera på datum, avd
df.sort_values(by=['datum','avd'],inplace=True)

# addera features
data = comp_new_features(df)

# ta bort alla features som inte kan användas och som jag inte vill ha
data = remove_features(data,remove_mer=[])
cat_features,int_features,float_features=feature_types(data.drop('datum',axis=1))
data=handle_cat_features(data,cat_features)

In [ ]:
results = estimator_loop(data,cat_features)

end = time.time()
delta = end - start
print(f'took {round(delta/60,2)} minutes to process')

In [ ]:
for x in ['Recall','F1','AUC','Precision','Accuracy']:
    list = [np.mean(results[x]),np.mean(results1[x]),np.mean(results2[x]),np.mean(results3[x]),np.mean(results4[x]),np.mean(results5[x]),np.mean(results6[x])]
    print(list.index(max(list)))
    
results    


## Kör en engångs körning utan iteration för att jämföra med PCA

In [ ]:
model = CatBoostClassifier(iterations= 1000, auto_class_weights= 'Balanced', 
            eval_metric= 'Recall', loss_function= 'Logloss', early_stopping_rounds= 100,
            random_seed= 2021)

alla_datum = df.datum.unique()
alla_datum = np.sort(alla_datum)
cut_ix = int(len(alla_datum)*0.75)
startvecka=alla_datum[cut_ix]
print('startvecka',startvecka)
print(len(alla_datum[cut_ix:]))
X_train = data[data.datum<alla_datum[cut_ix]].copy()
X_test = data[data.datum >= alla_datum[cut_ix]].copy()
y_train = (X_train.plac==1)*1;X_train.drop(['datum','plac'],axis=1,inplace=True)
y_test  = (X_test.plac==1)*1;X_test.drop(['datum','plac'],axis=1,inplace=True)
cat_features,int_features,float_features =feature_types(X_train)
# cat_features.remove('datum')

train_pool = Pool(X_train,y_train,cat_features=cat_features)
test_pool = Pool(X_test,y_test,cat_features=cat_features)
X_train.shape,X_test.shape,y_train.shape,y_test.shape
run_catB(model,train_pool, test_pool)


In [ ]:
#### JÄMFÖR RESULTAT MED PCA NEDAN  -  Slutsats: Skit i PCA!
dic=model.eval_metrics(test_pool,['AUC','Accuracy','F1','Recall','Precision'])
print(f"EJ PCA:\tAUC={round(np.max(dic['AUC']),4)}\tAcc={round(np.max(dic['Accuracy']),4)}\tF1={round(np.max(dic['F1']),4)} \t Recall={round(np.max(dic['Recall']),4)}\tPrecision={round(np.max(dic['Precision']),4)}")
confusion_matrix(y_test,model.predict(X_test))

# gammalt

In [ ]:
def catcv(X):
    X_train, X_test, y_train, y_test = split_data(X)
    cat_features, int_features, float_features = feature_types(X_train)
    # cat_features= []
    int_features=['kr', 'dist', 'lopp_dist', 'ålder']
    selected_features = cat_features + float_features  +int_features
    cv_pool = Pool(data=X_train[selected_features], label=y_train,cat_features=cat_features)
    params = {'iterations': 2000, 'auto_class_weights': 'Balanced', 
            'eval_metric': 'Accuracy', 'loss_function': 'Logloss', 'early_stopping_rounds': 200,
            'use_best_model': True, 'random_seed': 2021,
            }

    model = CatBoostClassifier( )
    scores = cv(cv_pool, params, shuffle = False, fold_count = 30, type='TimeSeries', verbose = 50 )
# catcv(X)

In [ ]:
def catfit(train_pool, test_pool):
    model = CatBoostClassifier(  
        iterations=2000, 
        auto_class_weights='Balanced',             
        eval_metric = 'Recall',
        loss_function = 'Logloss',
        early_stopping_rounds=100,
        random_seed=2021,
        )

    model.fit(train_pool,
        eval_set=test_pool,
        use_best_model=True,
        #plot=True,
        verbose = False,
    )

    return model


Utom 'vodds','podds','h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf'   
        AUC     F1  Accuracy  
mean: 0.8225 0.7805 0.7795  
std:  0.0887 0.0991 0.0827  
  
Utom 'vodds','podds','h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf', 'h1_perf'  
        AUC     F1  Accuracy  
mean: 0.8245 0.7777 0.7769  
std:  0.0920 0.1028 0.0849  

 utom 'vodds','podds'  
           AUC       F1    Accuracy    
mean     0.8242    0.7767    0.7759  
std      0.0889    0.1045    0.0876  

Utom 'vodds','podds','h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf'   
        AUC     F1  Accuracy  
mean: 0.8226 0.7756 0.7753  
std:  0.0897 0.1044 0.0869  

Utom 'vodds','podds','h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf', 'h1_perf','delta4'   
       AUC     F1  Accuracy  
mean  0.8206 0.7762 0.775  
std  0.09 0.1039 0.0865  

 

## PCA ###

In [ ]:
df.columns

In [ ]:
def omkodning(data,feat,mapTo):
    omk=data[feat].map(mapTo)
    omk.fillna(0,inplace=True)
    return omk

In [ ]:
df=pd.read_csv('all_data.csv')
####
import time
start = time.time()

# sortera på datum, avd
df.sort_values(by=['datum','avd'],inplace=True)

# addera features
X = comp_new_features(df)

# ta bort alla features som inte kan användas och som jag inte vill ha
X = remove_features(X,remove_mer=[])
cat_features,int_features,float_features=feature_types(X.drop('datum',axis=1))
handle_cat_features(data,cat_features)

### häst omkodning ###
mapTo = X.häst.value_counts().to_dict()
X['chäst']=omkodning(X,'häst',mapTo)
X.drop('häst',axis=1,inplace=True)

### kusk omkodning ###
kusk_dict = X.kusk.value_counts().to_dict()
X['ckusk']=X.kusk.map(kusk_dict)
X['ch1_kusk']=X.h1_kusk.map(kusk_dict)
# X['ch1_kusk'].fillna(0,inplace=True)
X['ch2_kusk']=X.h2_kusk.map(kusk_dict)
# X['ch2_kusk'].fillna(0,inplace=True)
X['ch3_kusk']=X.h3_kusk.map(kusk_dict)
# X['ch3_kusk'].fillna(0,inplace=True)
X['ch4_kusk']=X.h4_kusk.map(kusk_dict)
# X['ch4_kusk'].fillna(0,inplace=True)
X['ch5_kusk']=X.h5_kusk.map(kusk_dict)
# X['ch5_kusk'].fillna(0,inplace=True)
print(X.ch5_kusk.isna().sum())

X.drop(['kusk','h1_kusk','h2_kusk','h3_kusk','h4_kusk','h5_kusk'],axis=1,inplace=True)
### bana omkodning ###
bana_dict = X.bana.value_counts().to_dict()
X['cbana']=X.bana.map(bana_dict)
X['ch1_bana']=X.h1_bana.str.upper().map(bana_dict)
# X['ch1_bana'].fillna(0,inplace=True)
X['ch2_bana']=X.h2_bana.str.upper().map(bana_dict)
# X['ch2_bana'].fillna(0,inplace=True)
X['ch3_bana']=X.h3_bana.str.upper().map(bana_dict)
# X['ch3_bana'].fillna(0,inplace=True)
X['ch4_bana']=X.h4_bana.str.upper().map(bana_dict)
# X['ch4_bana'].fillna(0,inplace=True)
X['ch5_bana']=X.h5_bana.str.upper().map(bana_dict)
# X['ch5_bana'].fillna(0,inplace=True)

print(X.ch5_bana.unique(),'\n',X.ch5_bana.isna().sum())

X.drop(['bana','h1_bana','h2_bana','h3_bana','h4_bana','h5_bana'],axis=1,inplace=True)


In [ ]:
alla_datum = df.datum.unique()
alla_datum = np.sort(alla_datum)
cut_ix = int(len(alla_datum)*0.75)
startvecka=alla_datum[cut_ix]
print('startvecka',startvecka)
print(len(alla_datum[cut_ix:]))
print(f'train har {cut_ix} veckor av {len(alla_datum)}')
X_train = X[X.datum<alla_datum[cut_ix]].copy()
X_test = X[X.datum >= alla_datum[cut_ix]].copy()
y_train = (X_train.plac==1)*1;X_train.drop('plac',axis=1,inplace=True)
y_test  = (X_test.plac==1)*1;X_test.drop('plac',axis=1,inplace=True)
cat_features,int_features,float_features =feature_types(X_train)
cat_features.remove('datum')
X_train.shape,X_test.shape,y_train.shape,y_test.shape


In [ ]:
cat_features

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import gen_features
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

gen_numeric = gen_features(
    columns=[float_features,int_features],
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "mean"
        },
        {
            "class": StandardScaler
        }
    ]
)
print('gen_numeric',gen_numeric)
gen_gender = (
    ["kön","start"],
    [
        SimpleImputer(strategy="constant",fill_value='UNK'),
        OneHotEncoder()
    ],
)
from sklearn_pandas import DataFrameMapper

preprocess_mapper = DataFrameMapper(
    [
        gen_gender,
        *gen_numeric,
    ],
    input_df=True,
    df_out=True
)

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ("preprocess", preprocess_mapper),
    # ("estimator", RandomForestClassifier(n_estimators=100, max_depth=6,class_weight='balanced'))
    ("estimator", CatBoostClassifier(iterations=1000, 
        auto_class_weights='Balanced',             
        eval_metric = 'Accuracy',
        loss_function = 'Logloss',
        early_stopping_rounds=100,
        random_seed=2021,verbose=False))
])

pipeline.fit(X_train, y_train*1)
preds = pipeline.predict(X_test)
print('preds',preds)

In [ ]:
model=pipeline['estimator']
model.get_feature_importance(prettified=True)
# pipeline.get_params()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score,precision_score,recall_score
test_pool=Pool(preprocess_mapper.transform(X_test),label=y_test,cat_features=[])
preds=model.predict(test_pool)   #[:,0]
preds=preds*1
print (preds)
print(X_test.shape,y_test.shape,preds.shape)
print('Acc',round(accuracy_score(y_test,preds*1),4), 'F1',round(f1_score(y_test,preds*1),4),'Pre', round(precision_score(y_test,preds*1),4), 'Rec',round(recall_score(y_test,preds*1),4))
confusion_matrix(y_test,preds)

In [ ]:
(415)/(1095+415)

In [ ]:
def run_with_PCA(nr,X_train,X_test,y_train,y_test):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA

    # df.sort_values(by=['datum','avd'],inplace=True)
    # ta bort alla features som inte kan användas
    # X = df.drop(['avd','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1)
    # ta bort features som jag inte vill ha
    # X = X.drop(['vodds','podds'],axis=1)
    # X.dropna(inplace=True)
    # X.plac=(X.plac==1)*1
    # veckor = X.datum.unique() # blir en datum per vecka
    # startvecka = int(len(veckor) / 1.4)
    # X_train = X[X.datum<veckor[startvecka]].copy()
    # y_train = X_train.plac; X_train.drop(['plac','datum'],axis=1,inplace=True)
    # X_test = X[X.datum >=veckor[startvecka]].copy()
    # y_test = X_test.plac ; X_test.drop(['datum','plac'],axis=1,inplace=True)
          
    cat_features,int_features,float_features=feature_types(X_train)
    # X_train = X_train[int_features+float_features]
    # X_test = X_test[int_features+float_features]
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    pca = PCA(nr)

    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    explained_variance = pca.explained_variance_ratio_
    return X_train,X_test,y_train,y_test

In [ ]:
def get_model(X_train,X_test,y_train,y_test):
    train_pool = Pool(data=X_train, label=y_train,cat_features=[])
    test_pool  = Pool(data=X_test, label=y_test,cat_features=[])
    model = catfit(train_pool,test_pool) 
    return model,train_pool,test_pool
# model.get_feature_importance(prettified=True)


In [ ]:
#X_train.drop('datum',axis=1,inplace=True)
#X_test.drop('datum',axis=1,inplace=True)

for nr in [65,55,47,45,40,35,30,20,10,5]:
    pX_train=preprocess_mapper.fit_transform(X_train)
    pX_test=preprocess_mapper.transform(X_test)
    pX_train,pX_test,y_train,y_test=run_with_PCA(nr,pX_train,pX_test,y_train,y_test)
    model,train_pool,test_pool=get_model(pX_train,pX_test,y_train,y_test)
    dic=model.eval_metrics(test_pool,['AUC','Accuracy','F1','Recall','Precision'])

# print(np.max(dic['AUC']),np.max(dic['F1']),np.max(dic['Accuracy']))

    print(f"PCAnr={nr} \tAUC={round(np.max(dic['AUC']),4)}\tAcc={round(np.max(dic['Accuracy']),4)}\tF1={round(np.max(dic['F1']),4)} \t Recall={round(np.max(dic['Recall']),4)}\tPrecision={round(np.max(dic['Precision']),4)}")


EJ PCA:	AUC=0.7916	Acc=0.7342	F1=0.7517 	 Recall=0.8122	Precision=0.7228  
conf Matrix  
6111, 3197  
163,  705

In [ ]:
X = df.drop(['avd','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1)
# ta bort features som jag inte vill ha
X = X.drop(['vodds','podds'],axis=1)
X.dropna(inplace=True)
X.shape
    

In [ ]:
from catboost.utils import get_confusion_matrix
# print(model.predict_proba(test_pool)[:,0])
y_test.values
get_confusion_matrix(model,test_pool)

In [ ]:
print(f'len(X)-2: {len(X.columns)-2} \ncat: {cat_features}')


In [ ]:
test_df = df.iloc[:10,:10]
test_df['plac'] = (df.iloc[:10,:].plac==1)
test_df

In [ ]:
from sklearn_pandas import gen_features

numeric_features = [["avd"], ["streck"], ["vodds"], ["podds"], ["kr"],["spår"]]

gen_numeric = gen_features(
    columns=numeric_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "mean"
        },
        {
            "class": StandardScaler
        }
    ]
)

gen_numeric

In [ ]:
import sklearn
mapper = DataFrameMapper([
    (['vodds','podds','spår'], sklearn.preprocessing.StandardScaler()),
    ('plac', sklearn.preprocessing.LabelBinarizer()),
     
    ])

In [ ]:
mapper.fit_transform(test_df)

# sklearn.preprocessing.LabelBinarizer(test_df.plac)